In [1]:
import theano.sandbox.cuda

Using gpu device 2: Tesla K40m (CNMeM is disabled, cuDNN 5105)
/home/stud/shalaby/.virtualenv/thesis-env/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


In [2]:
from keras import backend

Using Theano backend.


/home/stud/shalaby/.virtualenv/thesis-env/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [3]:
import logging
from logging import info

In [4]:
root = logging.getLogger()
for handler in root.handlers[:]:
    root.removeHandler(handler)
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO) # adds a default StreamHanlder
#root.addHandler(logging.StreamHandler())

In [5]:
%%time
from theano import function, config, shared, sandbox
import theano.tensor as T
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], T.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, T.Elemwise) for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

[GpuElemwise{exp,no_inplace}(<CudaNdarrayType(float32, vector)>), HostFromGpu(GpuElemwise{exp,no_inplace}.0)]
Looping 1000 times took 0.919699 seconds
Result is [ 1.23178029  1.61879349  1.52278066 ...,  2.20771813  2.29967761
  1.62323296]
Used the gpu
CPU times: user 292 ms, sys: 668 ms, total: 960 ms
Wall time: 1.03 s


In [4]:
from keras.models import Sequential
from keras.layers import Dense
import numpy
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [5]:
# load pima indians dataset
dataset = numpy.loadtxt("pima-indians-diabetes.data", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, init='uniform', activation='relu'))
model.add(Dense(8, init='uniform', activation='relu'))
model.add(Dense(1, init='uniform', activation='sigmoid'))

In [7]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [8]:
# Fit the model
model.fit(X, Y, nb_epoch=150, batch_size=10)

Epoch 1/150
768/768 [==============================] - 0s - loss: 0.6826 - acc: 0.6328     
Epoch 2/150
768/768 [==============================] - 0s - loss: 0.6590 - acc: 0.6510     
Epoch 3/150
768/768 [==============================] - 0s - loss: 0.6475 - acc: 0.6549     
Epoch 4/150
768/768 [==============================] - 0s - loss: 0.6416 - acc: 0.6615     
Epoch 5/150
768/768 [==============================] - 0s - loss: 0.6216 - acc: 0.6745     
Epoch 6/150
768/768 [==============================] - 0s - loss: 0.6128 - acc: 0.6680     
Epoch 7/150
768/768 [==============================] - 0s - loss: 0.6018 - acc: 0.6927     
Epoch 8/150
768/768 [==============================] - 0s - loss: 0.5962 - acc: 0.6927     
Epoch 9/150
768/768 [==============================] - 0s - loss: 0.5991 - acc: 0.6953     
Epoch 10/150
768/768 [==============================] - 0s - loss: 0.5920 - acc: 0.6927     
Epoch 11/150
768/768 [==============================] - 0s - loss: 0.5905 - acc

In [13]:
# evaluate the model
scores = model.evaluate(X, Y)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

544/768 [====================>.........] - ETA: 0sacc: 78.91%


In [14]:
predictions = model.predict(X)

In [15]:
predictions

array([[ 0.82117373],
       [ 0.10473503],
       [ 0.9090184 ],
       [ 0.10512042],
       [ 0.78090107],
       [ 0.26003391],
       [ 0.16429627],
       [ 0.43324727],
       [ 0.94769281],
       [ 0.14849883],
       [ 0.21905506],
       [ 0.93988955],
       [ 0.5941481 ],
       [ 0.944345  ],
       [ 0.77902597],
       [ 0.31717807],
       [ 0.61684436],
       [ 0.32911414],
       [ 0.36678585],
       [ 0.22693451],
       [ 0.51870084],
       [ 0.43885848],
       [ 0.94429654],
       [ 0.34876618],
       [ 0.67694926],
       [ 0.54456681],
       [ 0.85718262],
       [ 0.09663744],
       [ 0.33522213],
       [ 0.28970671],
       [ 0.46944129],
       [ 0.67228943],
       [ 0.0961894 ],
       [ 0.02473637],
       [ 0.57788187],
       [ 0.47445977],
       [ 0.77614415],
       [ 0.6019994 ],
       [ 0.27857256],
       [ 0.69152761],
       [ 0.6963852 ],
       [ 0.74253017],
       [ 0.1207545 ],
       [ 0.81510013],
       [ 0.84786874],
       [ 0

In [16]:
rounded = [round(x) for x in predictions]

### Keras Embeddings

In [6]:
from keras.models import Sequential
from keras.layers import Dense, Embedding
import numpy as np

ImportError: cannot import name Dense

In [10]:
model = Sequential()
model.add(Embedding(1000, 64, input_length=10))
# the model will take as input an integer matrix of size (batch, input_length).
# the largest integer (i.e. word index) in the input should be no larger than 999 (vocabulary size).
# now model.output_shape == (None, 10, 64), where None is the batch dimension.

input_array = np.random.randint(1000, size=(32, 10))

model.compile('rmsprop', 'mse')
output_array = model.predict(input_array)
assert output_array.shape == (32, 10, 64)

In [11]:
input_array.shape

(32, 10)